In [4]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/Users/Jack/Documents/Projects/BigQueryPingPong/input/pingpong-322517-aab02d6e732c.json'

In [5]:
client = bigquery.Client()
# dataset_ref = client.dataset("PingPong", project="pingpong-322517")
# dataset = client.get_dataset(dataset_ref)

In [4]:
def head2head(client, player1, player2):
    '''
    Returns a head-to-head of the two specified players
    :param client: A Client object that specifies the connection to the dataset
    :param player1/player2: String, names of players to compare
    :return: Head-to-head of two players
    '''
    
    players = [player1, player2]
    players = sorted(players)
    player1, player2 = players[0], players[1]

    my_query = """
               WITH filter AS 
               (
               SELECT *
               FROM `pingpong-322517.PingPong.history`
               WHERE player1 = '{}' AND player2 = '{}'
               ORDER BY id
               )
               SELECT winner, COUNT(winner) AS wins
               FROM filter
               GROUP BY winner
               """.format(player1, player2)
    
    # Set up the query 
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results
    
head = head2head(client, 'Logan', 'Jack')
print(head)

  winner  wins
0   Jack     2
1  Logan    44


In [51]:
def rankings(client, display=False):
    """
    Returns player rankings with games >= 10
    :param client: A Client object that specifies the connection to the dataset
    :return: player rankings
    """
    if display:
        my_query = """
                SELECT *
                FROM `pingpong-322517.PingPong.current_rank`
                WHERE games >= 10
                ORDER BY rating DESC
                """
    else:
        my_query = """
                SELECT *
                FROM `pingpong-322517.PingPong.current_rank`
                ORDER BY rating DESC
                """
    
    # Set up the query 
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results
    
head = rankings(client)
print(head)

       name  rating  games
0     Logan    1388    278
1      Leif    1093     24
2      John    1045     43
3     Miler     937    172
4     Dario     936      6
5     Ahmed     933      7
6       Ben     929      3
7   Delaney     924      7
8     Henry     923    200
9     Tyler     913     12
10     Jack     897    146
11     Luis     874      9
12     Cade     865      9
13    Aidan     807     17
14  Murdoch     734     44


In [50]:
def history(client):
    '''
    Returns player ranking history
    :param client: A Client object that specifies the connection to the dataset
    :return: player ranking history 
    '''
    
    my_query = """
               SELECT *
               FROM `pingpong-322517.PingPong.ranking_history`
               ORDER BY id DESC
               """
    
    # Set up the query 
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results
    
hist = history(client)
print(hist)

      id  Logan  Leif  John  Miler  Dario  Ahmed   Ben  Delaney  Henry  Tyler  \
0    409   1386  1093  1045    937    936    933   929      924    923    913   
1    408   1386  1093  1045    937    936    933   929      924    923    913   
2    407   1381  1093  1051    937    936    933   929      924    923    913   
3    406   1376  1093  1057    937    936    933   929      924    923    913   
4    405   1370  1093  1064    937    936    933   929      924    923    913   
..   ...    ...   ...   ...    ...    ...    ...   ...      ...    ...    ...   
405    4   1000  1000     0   1000   1000    983  1000        0   1040      0   
406    3   1000  1000     0   1000   1000    960  1000        0   1040      0   
407    2   1000  1000     0   1000   1000    979  1000        0   1022      0   
408    1   1000  1000     0   1000   1000    979  1000        0   1000      0   
409    0   1000  1000     0   1000   1000   1000  1000        0   1000      0   

     Luis  Jack  Cade  Aida

In [72]:
df = rankings(client)
print(df)

      name  rating  games
0    Logan    1386    277
1     Leif    1093     24
2     John    1045     43
3    Miler     937    172
4    Henry     923    200
5    Tyler     913     12
6     Jack     900    144
7    Aidan     807     17
8  Murdoch     734     44


In [53]:
from rankingSystem import recordMatch
print(recordMatch(df, "Jack", "Logan", "Logan"))

NameError: name 'df' is not defined

In [49]:
def update_rankings(client, person1, person2, winner):
    
    df = rankings(client)
    new_ranking_1, new_ranking_2 = recordMatch(df, person1, person2, winner)

    my_query1 = """
                UPDATE `pingpong-322517.PingPong.current_rank`
                SET rating = {}, games = games + 1
                WHERE name = '{}'
                """.format(new_ranking_1,person1)
    my_query2 = """
                UPDATE `pingpong-322517.PingPong.current_rank`
                SET rating = {}, games = games + 1
                WHERE name = '{}'
                """.format(new_ranking_2,person2)

    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)   

    my_query_job1 = client.query(my_query1, job_config=safe_config)
    my_query_job1.result()
    my_query_job2 = client.query(my_query2, job_config=safe_config)
    my_query_job2.result()

    print(f"Rankings updated for game between {person1} and {person2}")


In [54]:
update_rankings(client, "Jack", "Logan", "Logan")
print(rankings(client))

Rankings updated for game between Jack and Logan
       name  rating  games
0     Logan    1390    279
1      Leif    1093     24
2      John    1045     43
3     Miler     937    172
4     Dario     936      6
5     Ahmed     933      7
6       Ben     929      3
7   Delaney     924      7
8     Henry     923    200
9     Tyler     913     12
10     Jack     894    147
11     Luis     874      9
12     Cade     865      9
13    Aidan     807     17
14  Murdoch     734     44


In [66]:
def update_rankings_goback(client, person1, person2, winner):
    df = rankings(client)
    new_ranking_1, new_ranking_2 = recordMatch(df, person1, person2, winner)
    print(new_ranking_1, new_ranking_2)

    my_query1 = """
                UPDATE `pingpong-322517.PingPong.current_rank`
                SET rating = 915, games = 145
                WHERE name = 'Jack'
                """
    my_query2 = """
                UPDATE `pingpong-322517.PingPong.current_rank`
                SET rating = 1386, games = 277
                WHERE name = 'Logan'
                """
                

    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)   

    my_query_job = client.query(my_query1, job_config=safe_config)
    my_query_job = client.query(my_query2, job_config=safe_config)

update_rankings_goback(client, "Jack", "Logan", "Logan")

950 1349


In [57]:
hist = history(client)
ranks = rankings(client)
ranks

,name,rating,games
0,Logan,1386,277
1,Leif,1093,24
2,John,1045,43
3,Miler,937,172
4,Dario,936,6
5,Ahmed,933,7
6,Ben,929,3
7,Delaney,924,7
8,Henry,923,200
9,Jack,915,145


In [27]:
# rank_mapping = dict(zip(ranks.name, ranks.rating))
# rank_mapping['id'] = hist.id.max() + 1
# names = hist.columns.to_list()
# # print(names)
# to_add = [rank_mapping.get(name) for name in names]
# # INSERT `sample-project-12345.Test.account` (`ID`, `Name`, `Salary` ) 
# # VALUES ('E001', 'John', 25000)
# my_query = """
#            INSERT INTO `pingpong-322517.PingPong.ranking_history` ()
#            FROM `pingpong-322517.PingPong.ranking_history`
#            """
# max_id = client.query(my_query).to_dataframe().iloc[0][0]


16

In [48]:
hist = history(client)
ranks = rankings(client)
def update_history(client, ranks, hist):

    names = ranks.name.tolist()
    names.append('id')
    columns_query = ', '.join(names)

    rank_vals = ranks.rating.tolist()
    rank_vals.append(hist.id.max() + 1)
    string_rank = [str(num) for num in rank_vals] 
    rank_query = ", ".join(string_rank) 

    my_query = f"""
                INSERT INTO `pingpong-322517.PingPong.ranking_history` ({columns_query})
                VALUES ({rank_query})
                """
    hist_update = client.query(my_query)
    hist_update.result()

    print(f"Rankings updated for row index {rank_vals[-1]}")

In [52]:
# update_history(client, ranks, hist)
print(history(client))

      id  Logan  Leif  John  Miler  Dario  Ahmed   Ben  Delaney  Henry  Tyler  \
0    409   1386  1093  1045    937    936    933   929      924    923    913   
1    408   1386  1093  1045    937    936    933   929      924    923    913   
2    407   1381  1093  1051    937    936    933   929      924    923    913   
3    406   1376  1093  1057    937    936    933   929      924    923    913   
4    405   1370  1093  1064    937    936    933   929      924    923    913   
..   ...    ...   ...   ...    ...    ...    ...   ...      ...    ...    ...   
405    4   1000  1000     0   1000   1000    983  1000        0   1040      0   
406    3   1000  1000     0   1000   1000    960  1000        0   1040      0   
407    2   1000  1000     0   1000   1000    979  1000        0   1022      0   
408    1   1000  1000     0   1000   1000    979  1000        0   1000      0   
409    0   1000  1000     0   1000   1000   1000  1000        0   1000      0   

     Luis  Jack  Cade  Aida

In [51]:
def delete_previous_history(client):
    hist = history(client)
    idx_to_delete = hist.id.max() 

    my_query =  f"""
                DELETE FROM `pingpong-322517.PingPong.ranking_history`
                WHERE id = {idx_to_delete}
                """

    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)   
    my_query_job = client.query(my_query, job_config=safe_config)
    print(f"Deleted column with id = {idx_to_delete}")

    

delete_previous_history(client)

Deleted column with id = 410


In [22]:
def game_history(client):
    my_query = """
               SELECT *
               FROM `pingpong-322517.PingPong.history`
               ORDER BY id DESC
               """
    game_history = client.query(my_query).to_dataframe()
    return game_history

In [24]:
def update_game_history(client, person1, person2, winner):
    my_query = """
               SELECT *
               FROM `pingpong-322517.PingPong.history`
               """
    
    game_hist = game_history(client)
    idx_to_add = game_hist.id.max() + 1
    
    update = f"{idx_to_add}, '{person1}', '{person2}', '{winner}'"
    my_update_query = F"""
                      INSERT INTO `pingpong-322517.PingPong.history` (id, player1, player2, winner)
                      VALUES ({update})
                      """
    game_history_update = client.query(my_update_query)
    game_history_update.result()

    print(f"Rankings updated for game between {person1} and {person2}")

In [25]:
update_game_history(client, "JACK", "LOGAN", "JACK")

Rankings updated for game between JACK and LOGAN


In [30]:
game_hist = game_history(client)
print(game_hist)

      id player1 player2 winner
0    350    John   Logan  Logan
1    349    John   Logan  Logan
2    348    John   Logan  Logan
3    347    John   Logan  Logan
4    346    John   Logan  Logan
..   ...     ...     ...    ...
346    4   Aidan   Henry  Henry
347    3   Ahmed   Aidan  Ahmed
348    2   Ahmed   Henry  Henry
349    1   Aidan   Henry  Henry
350    0   Ahmed   Aidan  Aidan

[351 rows x 4 columns]


In [29]:
def delete_previous_game(client):
    game_hist = game_history(client)
    idx_to_delete = game_hist.id.max() 

    my_query =  f"""
                DELETE FROM `pingpong-322517.PingPong.history`
                WHERE id = {idx_to_delete}
                """

    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)   
    my_query_job = client.query(my_query, job_config=safe_config)
    print(f"Deleted column with id = {idx_to_delete}")

delete_previous_game(client)

Deleted column with id = 351


In [55]:
def revert_back_rankings(client):

    # get newly updated game result
    recent_game = game_history(client)
    recent_game = recent_game.loc[recent_game['id'] == recent_game.id.max()]
    names_to_update = recent_game.values.tolist()[0][1:-1]

    # get old scores
    hist = history(client)
    idx_to_update = hist.id.max() - 1
    old_scores = hist.loc[hist['id'] == idx_to_update].to_dict('records')[0]

    my_query = f"""
               UPDATE `pingpong-322517.PingPong.current_rank`
               SET {names_to_update[0]} = {old_scores[names_to_update[0]]}, {names_to_update[1]} = {old_scores[names_to_update[1]]}
               WHERE condition
               """
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)   
    my_query_job = client.query(my_query, job_config=safe_config)
    print(f"Reverted scores")

revert_back_rankings(client)
print(rankings(client))

{'id': 408, 'Logan': 1386, 'Leif': 1093, 'John': 1045, 'Miler': 937, 'Dario': 936, 'Ahmed': 933, 'Ben': 929, 'Delaney': 924, 'Henry': 923, 'Tyler': 913, 'Luis': 874, 'Jack': 866, 'Cade': 865, 'Aidan': 807, 'Murdoch': 734}
Reverted scores
       name  rating  games
0     Logan    1390    279
1      Leif    1093     24
2      John    1045     43
3     Miler     937    172
4     Dario     936      6
5     Ahmed     933      7
6       Ben     929      3
7   Delaney     924      7
8     Henry     923    200
9     Tyler     913     12
10     Jack     894    147
11     Luis     874      9
12     Cade     865      9
13    Aidan     807     17
14  Murdoch     734     44


In [70]:
def revert_back_rankings(client):

    # get newly updated game result
    recent_game = game_history(client)
    recent_game = recent_game.loc[recent_game["id"] == recent_game.id.max()]
    names_to_update = recent_game.values.tolist()[0][1:-1]
    print(names_to_update)

    # get old scores
    hist = history(client)
    idx_to_update = hist.id.max() - 1
    old_scores = hist.loc[hist["id"] == idx_to_update].to_dict("records")[0]
    print(old_scores)
    my_query1 = f"""
               UPDATE `pingpong-322517.PingPong.current_rank`
               SET rating = {old_scores[names_to_update[0]]}, games = games - 1
               WHERE name = '{names_to_update[0]}'
               """
    print(my_query1)
    # my_query2 = f"""
    #         UPDATE `pingpong-322517.PingPong.current_rank`
    #         SET rating = {old_scores[names_to_update[1]]}, games = games - 1
    #         WHERE name = {names_to_update[1]}
    #         """
    # safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10 ** 10)
    # my_query_job = client.query(my_query1, job_config=safe_config)
    # my_query_job = client.query(my_query2, job_config=safe_config)
    # print(f"Reverted scores")
revert_back_rankings(client)


['Jack', 'Logan']
{'id': 410, 'Logan': 1386, 'Leif': 1093, 'John': 1045, 'Miler': 937, 'Dario': 936, 'Ahmed': 933, 'Ben': 929, 'Delaney': 924, 'Henry': 923, 'Tyler': 913, 'Luis': 874, 'Jack': 915, 'Cade': 865, 'Aidan': 807, 'Murdoch': 734}

               UPDATE `pingpong-322517.PingPong.current_rank`
               SET rating = 915, games = games - 1
               WHERE name = 'Jack'
               
